In [1]:
import numpy as np
import pandas as pd
import torch

pd.reset_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# test
x = torch.Tensor([1, 2, 3])
x
y = torch.Tensor([3, 4, 5])
z = (y - x) ** 2
z
z.sum()

tensor(12.)

In [3]:
feature = pd.read_excel(r'./input/feature.xls')
feature.shape

(6747, 132)

In [4]:
# extract effective feature
col = feature.columns[10:-1]
col



Index(['Leq_mean', 'Leq_var', 'Leq_std', 'Leq_max', 'Leq_10', 'Leq_25',
       'Leq_median', 'Leq_75', 'Leq_90', 'Leq_10-Leq_90',
       ...
       'sharpness_deviation2', 'tonality_tmax', 'tonality_phimax',
       'tonality_w', 'tonality_te', 'tonality_emd', 'tonality_esm',
       'tonality_slop1', 'tonality_slop2', 'tonality_deviation1'],
      dtype='object', length=121)

In [5]:
feature_use = feature[col]
feature_use.head()


,Leq_mean,Leq_var,Leq_std,Leq_max,Leq_10,Leq_25,Leq_median,Leq_75,Leq_90,Leq_10-Leq_90,...,sharpness_deviation2,tonality_tmax,tonality_phimax,tonality_w,tonality_te,tonality_emd,tonality_esm,tonality_slop1,tonality_slop2,tonality_deviation1
0,49.88,1.93,1.39,54.25,48.18,48.84,49.75,50.67,51.74,-3.56,...,0.22449,1.48699,0.12229,0.04647,3.34572,0.01873,0.75760,-0.28448,-0.41390,0.37501
1,49.71,2.68,1.64,54.70,47.90,48.37,49.31,50.83,52.21,-4.31,...,0.34575,4.50743,0.11179,0.04647,13.66171,0.02127,0.76335,0.68212,-0.54530,0.11233
2,50.53,9.36,3.06,55.87,46.26,48.45,51.20,52.79,53.95,-7.69,...,0.35291,9.47955,0.08939,0.13941,5.15799,0.01710,0.68687,0.09005,-0.88356,0.02650
3,49.96,13.89,3.73,57.87,45.00,46.82,50.18,52.87,55.17,-10.17,...,0.27142,0.46468,0.08878,0.04647,11.94238,0.02049,0.76139,0.13123,-0.13545,0.07406
4,50.51,3.25,1.80,58.05,48.56,49.16,50.09,51.52,52.70,-4.14,...,0.14822,1.20818,0.11093,0.04647,15.84572,0.02380,0.79167,-0.78949,-0.07725,0.23485


In [6]:
# Euclidean distance
def distEu(x, y):
    sum = np.sqrt(np.sum((x - y) ** 2))
    return sum

In [7]:
# A set of K random centers of mass
def randCenter(dataSet, k):
    # m represents row
    # n represents col
    m, n = dataSet.shape

    # initialize center
    centroid = np.zeros((k, n))
    centroid = pd.DataFrame(centroid)

    # traverse
    for i in range(k):
        # index为0-m之间的随机整数
        index = int(np.random.uniform(0, m))
        # calculate centroid and assign 
        centroid.iloc[i, :] = dataSet.iloc[index, :]
    return centroid


# k-means clustering
# k-means clustering
def KMeans(dataSet, k):
    m = np.shape(dataSet)[0]

    # 初始化一个矩阵来储存每个点的簇分配结果
    # clusterAssment包含两列：1列记录簇索引值，2列存储当前点到簇质心的距离（用来评价聚类的效果）
    clusterAssment = np.mat(np.zeros((m, 2)))
    clusterChange = True

    # 创建质心
    centroid = randCenter(dataSet, k)
    # 初始化标志变量，用于判断迭代是否继续，如果True，则迭代继续
    while clusterChange:
        clusterChange = False

        # 遍历所有行
        for i in range(m):
            minDist = 10000
            minIndex = -1
            # 遍历所有点找到距离每个点最近的质心
            for j in range(k):
                dist = distEu(centroid.iloc[j, :], dataSet.iloc[i, :])
                # 如果距离小于minDist，更新minDist和index索引值
                if dist < minDist:
                    minDist = dist
                    minIndex = j
            # 如果任意一点的簇分配结果发生改变，则更新clusterChanged标志
            if clusterAssment[i, 0] != minIndex:
                clusterChange = True
                # 更新簇分配结果为最小质心的index，minDist
                clusterAssment[i, :] = minIndex, minDist

        # 遍历所有质心并更新他们的取值
        for j in range(k):
            # 通过数据过滤来过的给定簇的所有点
            # .A表示把矩阵转化为array的形式
            # pointsInCluster = dataSet[np.nonzero(clusterAssment[:, 0].A == j)[0]]
            clusterAssment = pd.DataFrame(clusterAssment)
            same_cluster_index = clusterAssment.iloc[:, 0].isin([j])
            pointsInCluster = dataSet[same_cluster_index]

            # 计算所有点的均值，axis=0表示沿着列方向进行均值
            centroid.iloc[j, :] = np.mean(pointsInCluster, axis=0)
    print("Cluster complete!")
    return centroid, clusterAssment



In [17]:
k = 4
feature_test = feature_use.iloc[:20]
centroids, clusterAssment = KMeans(feature_test, k)


KeyboardInterrupt: 

In [9]:
centroids

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,54.1605,14.6675,3.0255,63.154,50.6615,51.873,53.8235,56.077,58.11,-7.4485,...,0.260277,4.36803,0.122435,0.123143,10.230018,0.020287,0.756458,-0.58334,-0.537472,0.215828
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
a = np.mat(np.zeros((3, 2)))
b = np.ones((3, 2))
a, b
a[2, :] = 1, 2
a
a.A

a[1, :] = b[2, :]
a

a, a[:, 0].A == 1, np.nonzero(a[:, 0].A == 1)

(matrix([[0., 0.],
         [1., 1.],
         [1., 2.]]),
 array([[False],
        [ True],
        [ True]]),
 (array([1, 2], dtype=int64), array([0, 0], dtype=int64)))

In [11]:
np.nonzero?

In [12]:
b2 = np.array([[True, False, True], [True, False, False]])
np.nonzero(b2)

a.astype(int)
a = pd.DataFrame(a)
flag = a.iloc[:, 0].isin([1])
a, flag, a[flag]

(     0    1
 0  0.0  0.0
 1  1.0  1.0
 2  1.0  2.0,
 0    False
 1     True
 2     True
 Name: 0, dtype: bool,
      0    1
 1  1.0  1.0
 2  1.0  2.0)

In [13]:
help(np.nonzero)

Help on function nonzero in module numpy:

nonzero(a)
    Return the indices of the elements that are non-zero.
    
    Returns a tuple of arrays, one for each dimension of `a`,
    containing the indices of the non-zero elements in that
    dimension. The values in `a` are always tested and returned in
    row-major, C-style order.
    
    To group the indices by element, rather than dimension, use `argwhere`,
    which returns a row for each non-zero element.
    
    .. note::
    
       When called on a zero-d array or scalar, ``nonzero(a)`` is treated
       as ``nonzero(atleast_1d(a))``.
    
       .. deprecated:: 1.17.0
    
          Use `atleast_1d` explicitly if this behavior is deliberate.
    
    Parameters
    ----------
    a : array_like
        Input array.
    
    Returns
    -------
    tuple_of_arrays : tuple
        Indices of elements that are non-zero.
    
    See Also
    --------
    flatnonzero :
        Return indices that are non-zero in the flattened 

In [20]:
a = np.zeros((3,4))
a
a= pd.DataFrame(a)
a

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [23]:
x = pd.DataFrame([1,2,3])
y = pd.DataFrame([3,4,5])
print(x)
print(y)
sum = np.sqrt(np.sum((x - y) ** 2))
sum

   0
0  1
1  2
2  3
   0
0  3
1  4
2  5


0    3.464102
dtype: float64

In [9]:
import numpy as np
def distEu(x, y):
    sum = np.sqrt(np.sum((x - y) ** 2))
    return sum

b = np.ones(15000)
c = np.zeros(15000)

import time
tic = time.time()*100000000
c = distEu(b, c)
toc = time.time()*100000000
toc - tic, c

(0.0, 122.47448713915891)

In [10]:
help(pd.concat)

Help on function concat in module pandas.core.reshape.concat:

concat(objs: Union[Iterable[ForwardRef('NDFrame')], Mapping[Optional[Hashable], ForwardRef('NDFrame')]], axis=0, join='outer', ignore_index: bool = False, keys=None, levels=None, names=None, verify_integrity: bool = False, sort: bool = False, copy: bool = True) -> Union[ForwardRef('DataFrame'), ForwardRef('Series')]
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes.
    
    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may be useful if the labels are the same (or overlapping) on
    the passed axis number.
    
    Parameters
    ----------
    objs : a sequence or mapping of Series or DataFrame objects
        If a mapping is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be
        selected (see below). Any None objects will be dropped silently unless
        